<b>Breaking Agent Deadlocks - algorithmic arbitration</b>

In [5]:
from IPython.display import Image

In [2]:
from pydantic import BaseModel, Field, ValidationError
import json

If you configure a "Coder Agent" prompted to write code quickly and a "Tester Agent" prompted to be extremely rigorous about finding edge cases, you have inadvertently designed an adversarial game. Mathematically, their individual reward functions ($R_c$ and $R_t$) are misaligned. They will quickly fall into a suboptimal Nash Equilibrium where the Coder refuses to rewrite the architecture and the Tester refuses to pass the build. Because LLMs are stateless autoregressive predictors, they will happily repeat the exact same argument for 50 API calls until your wallet is empty.To build a production-grade system, you cannot rely on the agents "working it out." <i>You must engineer strict Consensus Mechanisms and Deadlock Resolution patterns at the routing layer. </i> Here are the three architectural patterns you need to implement, moving from basic software engineering fail-safes to advanced algorithmic arbitration.

<b>Pattern 1: The Negotiation Token Budget (The "Time-to-Live" approach) </b> <br>
The simplest way to prevent an infinite loop is to stop using a global max_steps counter and start using Pairwise Negotiation Budgets. <br> 
Instead of letting agents converse freely, every edge in your multi-agent graph is assigned a finite budget $B$. Every time Agent A rejects Agent B's output, $B = B - 1$. <br>
<i> The Architecture: </i> In frameworks like AutoGen, this is partially implemented via max_consecutive_auto_reply. However, a true token budget is dynamic. If the Coder and Tester are making progress (the code diff is shrinking), the Manager dynamically increases $B$. If the code diff is identical for two turns (a detected loop), $B$ drops to 0 instantly. <br>
<i>The Resolution: </i> When $B = 0$, a TimeoutException is thrown to the Manager. The Manager forcefully terminates the sub-task, tags it as "Unresolved," and moves on, preventing the entire system from hanging.

<b>Pattern 2: The Arbitration Node (Hierarchical Tie-Breaking) </b> <br>
Timeouts are safe, but they result in failed tasks. To actually resolve the deadlock and complete the task, you need an Arbitration Agent (the "Judge").

The Architecture: You introduce a third agent whose sole job is anomaly resolution. It is not part of the standard workflow. It sits dormant until the Manager detects a deadlock (e.g., the Coder and Tester have gone back and forth 4 times).

<b>The Control Flow: </b> <br> 
1. The Manager intercepts the state, pausing both the Coder and Tester. <br>
2. The Manager bundles the last 4 messages (the argument) and passes them to the Arbitrator. <br>
3. The Arbitrator is prompted with a strict hierarchical override policy: "You are the Lead Engineer. The Coder and Tester are deadlocked. Read their argument. You must either (A) Force the Tester to accept the code with a documented warning, or (B) Force the Coder to implement a specific, overriding fix." <br>
4. The Arbitrator's output is injected into the state memory as a System Override, forcing the lower-level agents to comply. <br>

In [1]:
import hashlib

class DeadlockMonitor:
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries
        self.interaction_history = []
        
    def hash_message(self, message: str) -> str:
        """Creates a unique signature for an agent's argument."""
        return hashlib.md5(message.encode()).hexdigest()

    def detect_loop(self, latest_message: str) -> bool:
        """Checks if the agent is just repeating a previous argument."""
        msg_hash = self.hash_message(latest_message)
        
        # Count how many times we've seen this exact semantic response
        occurrences = self.interaction_history.count(msg_hash)
        self.interaction_history.append(msg_hash)
        
        if occurrences >= self.max_retries:
            return True
        return False

# Inside your Manager's Routing Loop:
deadlock_monitor = DeadlockMonitor(max_retries=2)

def run_multi_agent_thread(coder_output, tester_output):
    # ... worker execution ...
    
    if deadlock_monitor.detect_loop(tester_output):
        print("[SYSTEM] DEADLOCK DETECTED. Agents are repeating arguments.")
        
        # Trigger Pattern 2: The Arbitration Node
        resolution = call_arbitrator_agent(coder_output, tester_output)
        
        print(f"[ARBITRATOR OVERRIDE]: {resolution}")
        # Force the state forward, breaking the loop
        return resolution
        
    return tester_output